#### Simple Gen AI APP Using Langchain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [ ]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Lionel_Messi")
loader

In [ ]:
docs=loader.load()
docs

In [ ]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [ ]:
documents

In [ ]:
## Creating Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [ ]:
## Store embeddings in FAISS Vector DB
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstoredb

In [ ]:
## Query From a vector db when query text is not present in the documents stored in vector db
query="Narendra Modi"
result=vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
## Query From a vector db when query text is with the documents stored in vector db
query="Tell me about Lionel Messi's early career"
result=vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
## Create llm
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
## Create Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context, if you don't find any relevant information, say "I don't know".
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt) ## prompt here contains the context that is passed to the llm. 
document_chain

In [ ]:
## Invoke Document Chain by giving single docmument in the prompt
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Was there any democratic backsliding in India under Narendra Modi?", ## prompt input is not present in the documents stored in vector db, it should say "I don't know" as instructed in the prompt.
    "context":[ Document(page_content="Messi made his competitive debut for Barcelona at age 17 in October 2004. He gradually established himself as an integral player for the club, and during his first uninterrupted season at age 22 in 2008–09 he helped Barcelona achieve the first treble in Spanish football. This resulted in Messi winning the first of four consecutive Ballon d'Ors, and by the 2011–12 season he set the European record for most goals in a season and established himself as Barcelona's all-time top scorer. During the 2014–15 campaign, where he became the all-time top scorer in La Liga, he led Barcelona to a historic second treble, leading to a fifth Ballon d'Or in 2015. He assumed Barcelona's captaincy in 2018 and won a record sixth Ballon d'Or in 2019. At Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four Champions Leagues, among others. Financial difficulties at Barcelona led to Messi signing with French club Paris Saint-Germain in August 2021, where he won the Ligue 1", metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'}),]
})

In [ ]:
## Change the prompt template to include information if context is present
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context".
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt) 
document_chain
## Invoke Document Chain by giving single docmument in the prompt
document_chain.invoke({
    "input":"When did Messi make his competitive debut ", ## prompt input is present in the documents stored in vector db, it should return proper response.
    "context":[ Document(page_content="Messi made his competitive debut for Barcelona at age 17 in October 2004. He gradually established himself as an integral player for the club, and during his first uninterrupted season at age 22 in 2008–09 he helped Barcelona achieve the first treble in Spanish football. This resulted in Messi winning the first of four consecutive Ballon d'Ors, and by the 2011–12 season he set the European record for most goals in a season and established himself as Barcelona's all-time top scorer. During the 2014–15 campaign, where he became the all-time top scorer in La Liga, he led Barcelona to a historic second treble, leading to a fifth Ballon d'Or in 2015. He assumed Barcelona's captaincy in 2018 and won a record sixth Ballon d'Or in 2019. At Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four Champions Leagues, among others. Financial difficulties at Barcelona led to Messi signing with French club Paris Saint-Germain in August 2021, where he won the Ligue 1", metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'}),]
})